In [ ]:
directions = [45,35,25,15,5]
labels = ['+20','+10','training','-10','-20']
labeldict = dict(zip(directions,labels))
n_each = 15

In [ ]:
import random
import pandas as pd
import numpy as np

In [ ]:
dirs = []
lastdir = None
for _ in range(n_each):
    acceptable = False
    while not acceptable:
        d = directions[:]
        random.shuffle(d)
        acceptable = d[0]!=lastdir
    lastdir=d[-1]
    dirs+=d

In [ ]:
" ".join([str(d) for d in dirs ])

In [ ]:
tab = pd.DataFrame({'direction':dirs})
tab['type']=[ labeldict[d] for d in dirs ]
tab['trial']=np.arange(len(dirs))+1
tab.to_csv('recognitionyesno.csv')

In [ ]:
tab